# Constrained Dynamics

> A *constraint* is a modification of a dynamical system to maintain the consistency of a given function of coordinates. A constraint is *holonomic* if it can be described only in terms of position and time.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

## Vibration

The Hamiltonian
$$H = p^2/2m + U(q) + \epsilon^{-1}(q - r_0)^2/2$$
has the equations of motion,
$$\begin{split}
\dot{q} &= p/m\\
\dot{p} &= -\nabla U(q) - \nabla\phi_\epsilon(q),
\end{split}$$
where $\phi_\epsilon(q) = \epsilon^{-1}(\vert q\vert - r_0)^2/2$.

### Harmonic potential

Under a harmonic potential $U(q) = q^2/2$, the corresponding Hamiltonian with unit mass is
$$H = p^2/2 + q^2/2 + \epsilon^{-1}(q - r_0)^2/2.$$

In [ ]:
def hamiltonian(q, p, epsilon, r0):
    return p**2/2 + q**2/2 + (q - r0)**2/(2*epsilon)

qq = np.linspace(-1, 1, 100)
pp = np.linspace(-1, 1, 100)

Q, P = np.meshgrid(qq, pp)
H = hamiltonian(Q, P, 1, 0)

In [ ]:
plt.contourf(Q, P, H)
plt.show()

The corresponding equations of motion are
$$\begin{split}
\dot{q} &= p\\
\dot{p} &= -q - \epsilon^{-1}(q - r_0)\end{split}$$

### Velocity Verlet

Consider a splitting of the equations of motion,
$$\begin{split}
\mathcal{U}_h^Q &= [q + hM^{-1}p, p]\\
\mathcal{U}_h^P &= [q, p - h\nabla U(q)],
\end{split}$$
then the velocity verlet can be interpreted as the splitting scheme $[\![PQP]\!]$. In the specific case above,
$$\begin{split}
\mathcal{U}_h^Q &= [q + hp, p]\\
\mathcal{U}_h^P &= [q, p - h(q + \epsilon^{-1}(q - r_0))]
\end{split}$$

In [ ]:
def solve(initial_state, tt, epsilon, r0):
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    h = (t_max - t_min) / (n - 1)
    
    qq = [None for _ in range(n)]
    pp = [None for _ in range(n)]
    
    for i in range(n):
        if i == 0:
            qq[0] = initial_state[0]
            pp[0] = initial_state[1]
            pass
        else:
            q_curr = qq[i-1]
            p_curr = pp[i-1]
            
            _p = p_curr - h/2*(q_curr + (q_curr - r0) / epsilon)
            q_new = q_curr + h*_p
            p_new = _p - h/2*(q_new + (q_new - r0) / epsilon)
            
            qq[i], pp[i] = q_new, p_new
            pass
        pass
    return qq, pp

In [ ]:
initial_state = [1, 0]
n = 1000
tt = np.linspace(0, 10, n)

Investigate effects of altering $\epsilon$ and set $r_0 = 0$.

In [ ]:
m = 1000
epsilons = np.linspace(0, 1, m)

solution = [None for _ in range(m)]
for j in range(m):
    solution[j] = solve(initial_state, tt, epsilons[j], 0)[0]
    pass

solution = np.array(solution)
T, E = np.meshgrid(tt, epsilons)

In [ ]:
plt.contourf(E, T, solution)
plt.xlabel('$\epsilon$')
plt.ylabel('$t$')
plt.show()

Investigate effects of $r_0$ and setting $\epsilon = .1$.

In [ ]:
m = 1000
rr = np.linspace(-1, 1, m)

solution = [None for _ in range(m)]
for j in range(m):
    solution[j] = solve(initial_state, tt, .1, rr[j])[0]
    pass

solution = np.array(solution)
T, R = np.meshgrid(tt, rr)

In [ ]:
plt.contourf(R, T, solution)
plt.xlabel('$r$')
plt.ylabel('$t$')
plt.show()

Setting $q = 1$, $p = 0$ and varying $\epsilon$ and $r_0$.

In [ ]:
epsilons = np.linspace(.1, 1, 100)
rr = np.linspace(-1, 1, 100)

E, R = np.meshgrid(epsilons, rr)
H = hamiltonian(1, 0, E, R)

In [ ]:
plt.contourf(E, R, H)
plt.xlabel('$\epsilon$')
plt.ylabel('$r_0$')
plt.show()

### Euler Lagrange equations

Let the constraint in question be $g(q) = q^2 - r_0^2$, then the constrained equations of motion are
$$\begin{split}
\dot{q} &= p,\\
\dot{p} &= -q - \lambda g'(q),\\
0 &= g(q).
\end{split}$$
Taking a time derivative, $\dfrac{dg}{dt}(q) = 2qp = 0$ is our hidden constraint. Thus, the set of points $(q, p)$ satisfying the constraints,
$$\begin{split}
q^2 - r_0^2 &= 0\\
qp &= 0,
\end{split}$$
is our co-tangent bundle $T^*\mathcal{M}$. When $r_0\neq 0$, our solutions are $q^2 = r_0^2$ and $p = 0$.

Consider a constrained symplectic method,
$$\begin{split}
q_{n+1} &= q_n + hp_{n+1}\\
p_{n+1} &= p_n + hq_n - hq_np_n\lambda_n\\
0 &= g(q_{n+1})
\end{split}$$
We want the equation
$$g(Q_n - G_n\Lambda) = 0,$$
where $Q_n = q_n + hp_n + h^2q_n$, $G_n = 2q_np_n$ and $\Lambda = h^2\lambda$, satisfied. Therefore
$$(Q_n - G_n\Lambda)^2 = r_0^2,$$
and $Q_n - G_n\Lambda = \mathrm{sgn}[Q_n - G_n\Lambda]\mathrm{sgn}[r_0]r_0$. Solving for $\lambda_n$ yields
$$\lambda_n = \dfrac{q_n + hp_n + hq_n^2 - \mathrm{sgn}[r_0(Q_n - G_n\Lambda)]r_0}{2h^2q_np_n}.$$